In [106]:
# 导入包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [113]:
# 读取数据
!ls data/

df_train = pd.read_csv("data\\first_round_training_data.csv")

# 去掉多余的特征
#df_train = df_train.drop(df_train.columns[[10,11,12,13,14,15,16,17,18,19,]], axis=1, )
df_train = df_train.drop(df_train.columns[[10,11,12,13,14,15 , 16,17,18,19]], axis=1, )


# 特征10 换位到最后一列
df_test = pd.read_csv("data\\first_round_testing_data.csv")
tmp = df_test["Parameter10"].copy()
df_test.drop("Parameter10",axis=1, inplace=True)
df_test["Parameter10"] = tmp

# 制作标签
def makeLabel(row):
    #print(row)
    if row["Quality_label"] == "Fail":
        return 0
    elif row["Quality_label"] == "Pass":
        return 1
    elif row["Quality_label"] == "Good":
        return 2
    elif row["Quality_label"] == "Excellent":
        return 3

df_train["Quality_label"] = df_train["Quality_label"].map({"Excellent":3,"Good":2,"Pass":1,"Fail":0}) 
df_train.head()



# 划分数据集
df_target = df_train["Quality_label"]
train = df_train.drop(["Quality_label"], axis=1)


X_train,X_test, y_train, y_test = train_test_split(train,df_target,test_size=0.3, random_state=0)
### model-4 xgboost
import xgboost as xgb

score_train = []
score_test = []
c_list = [i for i in range(1, 10,1)]
#c_list = []
#for c in c_list:
    
reg = xgb.XGBClassifier(
      max_depth=5    # step2
    , learning_rate=0.1
    , n_estimators=29  # step1
    , verbosity=1
    , silent=None
    , objective="binary:logistic"
    , booster='gbtree'
    , n_jobs=1
    , nthread=None
    , gamma=0
    , min_child_weight=5   # step3

    , max_delta_step=0
    , subsample=1
    , colsample_bytree=0.7
    , colsample_bylevel=1
    , colsample_bynode=1
    , reg_alpha=0.3
    , reg_lambda=1
    , scale_pos_weight=1
    , base_score=0.5
    , random_state=2019
    , seed=None
    , missing=None
    , importance_type="gain"
    )
reg = reg.fit(X_train, y_train)


# train_score = reg.score(X_train, y_train)
# score_train.append(train_score)

# test_score = reg.score(X_test, y_test)
# score_test.append(test_score)
    
# plt.figure(figsize=[8,4])
# plt.plot(c_list,score_test)
# print(max(score_train))
# print(max(score_test))
# print(c_list[list(score_test).index(max(list(score_test)))])
reg.get_booster().save_model('xgb.model')
# model = xgb.XGBClassifier(max_depth=5, n_estimators=200, learn_rate=0.01)
# model.fit(X_train, y_train)  
# test_score = model.score(X_test, y_test)
# print('test_score: {0}'.format(test_score))


a.py
first_round_testing_data.csv
first_round_training_data.csv


In [114]:
def score(res, target):
    if len(res) != len(target):
        raise Exception("数据错误")
    
    trueNum = 0
    for i in range(len(res)):
        if res[i] == y_train.iloc[i]:
            trueNum += 1
    print(float(trueNum/len(res)))

In [115]:
# 划分数据集
df_target = df_train["Quality_label"]
train = df_train.drop(["Quality_label"], axis=1)


X_train,X_test, y_train, y_test = train_test_split(train,df_target,test_size=0.3, random_state=0)
### model-4 xgboost
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)


model = xgb.XGBClassifier(max_depth=5, n_estimators=200, learn_rate=0.01)
model.fit(X_train, y_train)  
test_score = model.score(X_test, y_test)
print('test_score: {0}'.format(test_score))

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#对数据的训练集进行标准化

# x = X_train.copy()
# ss = StandardScaler()
# X= ss.fit_transform(x)


## model -1 逻辑回归
# clf = LogisticRegression(random_state=2, solver='lbfgs',multi_class='multinomial',max_iter = 1000).fit(X, y_train)
# print(clf.score(X,y_train))


### model -2  决策树 

# from sklearn.tree import DecisionTreeClassifier

# treeClf = DecisionTreeClassifier(
#                                      max_depth=7
#                                    # ,splitter = "random"
#                                     ,min_samples_split = 40
#                                     ,random_state = 6
    
#                                         )
# treeClf = treeClf.fit(X_train, y_train)
# res = treeClf.score(X_test,y_test)
# print(res)

### model-3  随机森林  # 最好成绩 0.44
# from sklearn.ensemble import RandomForestClassifier

# rfc = RandomForestClassifier(
#                           n_estimators=100  # 默认10，决策树的数量, 越多越好，从100开始。
                          
#                         , bootstrap=True   # 有放回采样
#                         , oob_score=False  # out-of-bag 使用没有抽到的数据做验证
#                         , n_jobs=None      #并行计算
#                         , verbose=0        # 打印训练日志
#                         , warm_start=False  # 热启动
                        
#                         #与决策树相关的参数
#                         , criterion='gini'  # 或 "entropy" 
#                         , random_state=None
#                         , class_weight=None
#                         , max_depth=None
#                         , min_samples_split=2
#                         , min_samples_leaf=1
#                         , min_weight_fraction_leaf=0.0
#                         , max_features='auto' # auto=特征数的开根号，log2=log2(n_features)， None = n_features
#                         , max_leaf_nodes=None
#                         , min_impurity_decrease=0.0
#                         , min_impurity_split=None
#                         )

# rfc = rfc.fit(X_train, y_train)
# score = rfc.score(X_test,y_test)
# print(score)





test_score: 0.49944444444444447


In [116]:
### 调参过程

# score_list = []
# for c in range(340,380,2):
#     model = xgb.XGBClassifier(
#           max_depth=5
#         , learning_rate=0.1
#         , n_estimators=c   # step1: 360
#         , verbosity=1
#         , silent=None
#         , objective='reg:squarederror'
#         , booster='gbtree'
#         , min_child_weight=1

#         , scale_pos_weight=1
#         , base_score=0.5
#         , random_state=0
#         , seed=None
#         , missing=None
#         , importance_type='gain',
        
#         )
#     model.fit(X_train, y_train)  
#     test_score = model.score(X_test, y_test)
#     #print('test_score: {0}'.format(test_score))
#     score_list.append(test_score)
# print("best n_estimators: ", range(340,380,2)[score_list.index(max(score_list))])
# print("best score:", max(score_list))
# plt.figure(figsize=[8,4])
# plt.plot(range(340,380,2),score_list)

from sklearn.model_selection import GridSearchCV

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test,label=y_test)
cv_params = {'random_state': [42,2,3,2019,2000,10,1,6666]}
other_params = {    
                   # step1: 24
                    "n_estimators":200
    
                  # step2: 4, 3
                    , "max_depth":4
                    , "min_child_weight" : 3
    
                  # step3:
                    , "subsample" : 0.7
                    , "colsample_bytree" : 1
    
                
                    , "learning_rate":0.1
                
                    , "verbosity":1
                    , "silent":None
                    , "objective":'binary:logistic'
                    , "booster": "gbtree"
                    , "n_jobs" : 1
                    , "nthread" : None
                    , "gamma" : 0
                  
                    , "max_delta_step" : 0

                    , "colsample_bylevel" : 1
                    , "colsample_bynode" : 1
                    , "reg_alpha" : 0
                    , "reg_lambda" : 1
                    , "scale_pos_weight" : 1
                    , "base_score" : 0.5
                  #  , "random_state" : 42
                    , "seed"  : None
                    , "missing" : None
                    , "importance_type" :'gain'
}

model = xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=1)
optimized_GBM.fit(train, df_target)

evalute_result = optimized_GBM.cv_results_['mean_test_score']
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

# 0.5118

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  2.1min finished


每轮迭代运行结果:[0.49583333 0.4985     0.5005     0.50266667 0.49383333 0.498
 0.49816667 0.49883333]
参数的最佳取值：{'random_state': 2019}
最佳模型得分:0.5026666666666667


In [117]:
#
model = xgb.XGBClassifier(learning_rate=0.1, n_estimators=24, max_depth=4, min_child_weight=3, seed=0,
                         subsample=0.7, colsample_bytree=1, gamma=0, reg_alpha=0, reg_lambda=1)
model.fit(train, df_target)



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing=None, n_estimators=24, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              silent=None, subsample=0.7, verbosity=1)

In [118]:
# 测试集
testRes = model.predict(df_test.iloc[:,1:-1])

testpredict = model.predict_proba(df_test.iloc[:,1:-1])


ValueError: feature_names mismatch: ['Parameter1', 'Parameter2', 'Parameter3', 'Parameter4', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10'] ['Parameter1', 'Parameter2', 'Parameter3', 'Parameter4', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9']
expected Parameter10 in input data

In [ ]:
y_pred = pd.DataFrame(
    testpredict, columns=["Excellent ratio", "Good ratio", "Pass ratio", "Fail ratio"]
)
y_pred["Group"] = df_test["Group"]

group_pred = y_pred.groupby("Group").mean()
for c in ["Excellent ratio", "Good ratio", "Pass ratio", "Fail ratio"]:
    group_pred[c] /= group_pred.sum(axis=1)
    

group_pred.to_csv('./result1.csv', index=True, header=True)
group_pred.head()

In [ ]:
# 类别结果加到原始的测试集
df_test = pd.concat([df_test, pd.DataFrame(data=testRes)], axis=1,)

In [ ]:
submit_csv = pd.read_csv("submit_example.csv")
submit_csv.head()

In [ ]:
# 分组计算 概率
g = df_test.groupby("Group")
group_list = df_test.Group.unique()

for i in group_list:
    tmp = g.get_group(i)
    Excellent = len(tmp[tmp[0]==3])/50.0
    Good = len(tmp[tmp[0]==2])/50.0
    Pass = len(tmp[tmp[0]==1])/50.0
    Fail = len(tmp[tmp[0]==0])/50.0
    submit_csv[submit_csv["Group"]==i] = [i, Excellent, Good, Pass, Fail]

In [ ]:

submit_csv.to_csv('./result.csv', index=False, header=True)